In [ ]:
import pandas as pd
import pymysql.connector as mariadb
from pymongo import MongoClient
from sqlalchemy import create_engine
from config.constants import LEAGUES_DATA_DICT, CSV_EXPORT_PATH, CSV_EXPORT_PATH_MERGED, IDS_FILE_PATH
from converters.data2frames import read_json
import os
from tqdm import tqdm
import numpy as np

#  MongoDB

In [77]:
client2 = MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [80]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [81]:
db = client2.slds

In [82]:
m_coll = db.soloq_m
tl_coll = db.soloq_tl

In [7]:
def __get_file_names_from_match_id(m_id, save_dir):
    file_names = os.listdir(save_dir)
    match_filename = [val for val in file_names if str(m_id) in val and 'tl' not in val][0]
    tl_filename = [val for val in file_names if str(m_id) in val and 'tl' in val][0]
    return {'match_filename': match_filename.split('.')[0], 'tl_filename': tl_filename.split('.')[0]}

In [9]:
LEAGUES_DATA_DICT['SOLOQ']

{'csv_export_path': '../exports/soloq_dataset.csv',
 'csv_export_path_merged': '../exports/soloq_dataset_merged.csv',
 'dtypes': {},
 'excel_export_path': '../exports/soloq_dataset.xlsx',
 'excel_export_path_merged': '../exports/soloq_dataset_merged.xlsx',
 'ids_file_path': '../leagues_data/soloq.csv',
 'official_league': False,
 'raw_data_path': '../matches_raw_data/soloq/'}

In [21]:
read_dir = LEAGUES_DATA_DICT['SOLOQ']['raw_data_path']
f_names = os.listdir(read_dir)
m_ids = [name.split('.')[0].split('_')[1] for name in f_names if 'tl' not in name]

In [50]:
for g_id in tqdm(m_ids):
    f1 = read_json(save_dir=read_dir, file_name=__get_file_names_from_match_id(m_id=g_id, save_dir=read_dir)['tl_filename'])
    f1['gameId'] = int(g_id)
    tl_coll.insert_one(f1)

100%|████████████████████████████████████████████████████████████████████████████| 14662/14662 [24:05<00:00, 10.08it/s]


In [83]:
cursor = m_coll.find({}, {'_id': 0, 'gameId': 1})

In [84]:
l1 = [gid['gameId'] for gid in cursor]

In [85]:
len(l1)

14662

# MariaDB

In [ ]:
df = pd.read_excel(LEAGUES_DATA_DICT['SOLOQ']['excel_export_path_merged']).drop('summonerName', axis=1)

In [23]:
df2 = df.drop('summonerName', axis=1) 

In [86]:
engine = create_engine('mysql+mysqlconnector://davidherasp:.@127.0.0.1/exports')

In [25]:
id_list = [num for num in df2.gameId.unique()]

In [26]:
for num in id_list:
    df3 = df2.loc[df.gameId == num]
    df3.to_sql('soloq', engine, if_exists='append')

In [9]:
SQL_SOLOQ_CONN = {
    'user': 'davidherasp',
    'password': '.',
    'host': '127.0.0.1',
    'database': 'leagues_info'
}

In [69]:
cnx = mariadb.connect(**SQL_SOLOQ_CONN)

In [70]:
cursor = cnx.cursor()

In [84]:
name = 'MAD'.split(',')

In [86]:
query = ('SELECT DISTINCT account_id FROM soloq WHERE team_abbv = {}'.format('\"' + name[0] + '\"'))

In [87]:
cursor.execute(query)

In [88]:
l1 = [gid[0] for gid in cursor]

In [89]:
l1

[219693852,
 30080667,
 232115809,
 235096257,
 200733551,
 228412763,
 23224919,
 35915616,
 79725,
 229426824]

# One team dataset

In [2]:
df = pd.read_csv(LEAGUES_DATA_DICT['SOLOQ'][CSV_EXPORT_PATH], index_col=0, encoding='ISO-8859-1')

C:\Users\david\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (47,48,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
accs = pd.read_csv(LEAGUES_DATA_DICT['SOLOQ'][IDS_FILE_PATH], index_col=0, encoding='ISO-8859-1')

In [4]:
team_accs = list(accs.loc[accs.team_abbv.isin(['OG', 'GO', 'IHG'])].account_id)

In [10]:
df.dropna(subset=['currentAccountId'], inplace=True)

In [11]:
df.currentAccountId = df.currentAccountId.astype(np.int64)

In [13]:
df.currentAccountId.unique()

array([227343777, 231579807, 204715156, ...,  25205997, 238100511,
        22507694], dtype=int64)

In [14]:
df2 = df.loc[df.currentAccountId.isin(team_accs)]

In [15]:
df2.shape, df.shape

((2633, 213), (201118, 213))

In [17]:
df3 = df.loc[df.gameId.isin(df2.gameId)]

In [22]:
df3.shape, len(df2.gameId.unique())

((22390, 213), 2239)

In [23]:
df2.gameCreation.max()

'2018-04-26 02:42:15'

In [24]:
df2.to_excel('ahg_og_go_soloq_not_merged.xlsx')

In [25]:
df2.to_csv('ahg_og_go_soloq_not_merged.csv')